In [ ]:
import json
import tensorflow as tf

label_file = open("./imagenet-sample/labels.json")
label_file = json.load(label_file)

img_to_label = label_file['labels']
img_names = list(img_to_label.keys())

In [2]:
i = 0
#Define the epsilon constant (L_inf bound)
EPS = 2

#To constric the GPU choice incase some GPUs are already in use
device = "/device:GPU:1"
gpus = tf.config.list_physical_devices('GPU')

tf.config.set_visible_devices(gpus[4], 'GPU')

In [ ]:
# import necessary packages
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.applications.resnet50 import decode_predictions
from tensorflow.keras.applications.resnet50 import preprocess_input
import tensorflow as tf
import numpy as np
import argparse
import cv2


def preprocess_image(image):
	# swap color channels, resize the input image, and add a batch
	# dimension
	image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
	image = cv2.resize(image, (224, 224))
	image = np.expand_dims(image, axis=0)
	# return the preprocessed image
	return image

def clip_eps(tensor, eps):
	# clip the values of the tensor to a given range and return it
	return tf.clip_by_value(tensor, clip_value_min=-eps,
		clip_value_max=eps)

def generate_adversaries(model, baseImage, delta, classIdx, steps=50):
	# iterate over the number of steps
	for step in range(0, steps):
		# record our gradients
		with tf.GradientTape() as tape:
			# explicitly indicate that our perturbation vector should
			# be tracked for gradient updates
			tape.watch(delta)
			# add our perturbation vector to the base image and
			# preprocess the resulting image
			adversary = preprocess_input(tf.clip_by_value(baseImage + delta, 0, 255))
			# run this newly constructed image tensor through our
			# model and calculate the loss with respect to the
			# *original* class index
			predictions = model(adversary, training=False)
			loss = -sccLoss(tf.convert_to_tensor([classIdx]), predictions)
			# calculate the gradients of loss with respect to the
			# perturbation vector
			gradients = tape.gradient(loss, delta)
			# update the weights, clip the perturbation vector, and
			# update its value
			optimizer.apply_gradients([(gradients, delta)])
			delta.assign(clip_eps(delta, eps=EPS))
	# return the perturbation vector
	return delta

# define the learning rate constant
LR = 0.1

perturbation_list = []

ind = 0
for img in img_names:
	if(ind!=i):
		ind+=1
		continue
	i=i+1
	ind=ind+1
	print("Image : "+str(i))
	# load the input image from disk and preprocess it
	image = cv2.imread('./imagenet-sample/data/'+img+'.jpg')
	image = preprocess_image(image)

	# load the pre-trained ResNet50 model for running inference
	# print("[INFO] loading pre-trained ResNet50 model...")
	model = ResNet50(weights="imagenet")
	# initialize optimizer and loss function
	optimizer = Adam(learning_rate=LR)
	sccLoss = SparseCategoricalCrossentropy()

	# create a tensor based off the input image and initialize the
	# perturbation vector (we will update this vector via training)
	baseImage = tf.constant(image, dtype=tf.float32)
	delta = tf.Variable(tf.zeros_like(baseImage), trainable=True)
	# generate the perturbation vector to create an adversarial example
	deltaUpdated = generate_adversaries(model, baseImage, delta, img_to_label[img])
	
	perturbation_list.append(deltaUpdated)
	np.save('perturbations.npy',np.array(perturbation_list))
	
	# create the adversarial example, swap color channels, and save the
	# output image to disk
	adverImage = (baseImage + deltaUpdated).numpy().squeeze()
	adverImage = np.clip(adverImage, 0, 255).astype("uint8")
	adverImage = cv2.cvtColor(adverImage, cv2.COLOR_RGB2BGR)
	cv2.imwrite('./adv_png/'+img+'_adversarial.png', adverImage)

In [ ]:
import numpy as np
perturbation_list = np.load('perturbations.npy')
print(np.min(perturbation_list))
print(np.max(perturbation_list))
perturbation_list = []